In [406]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [407]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
649,tt0960731,80000000,212874442,Bedtime Stories,Adam Sandler|Keri Russell|Guy Pearce|Courteney...,Adam Shankman,Whatever they dream up... he has to survive.,Skeeter Bronson is a down-on-his-luck guy who'...,99,Fantasy|Comedy|Family|Romance,Walt Disney Pictures|Happy Madison Productions...,12/24/2008,5.8,2008
839,tt1742650,24000000,30551495,I Don't Know How She Does It,Sarah Jessica Parker|Pierce Brosnan|Sarah Shah...,Douglas McGrath,"If it were easy, men would do it too.","A comedy centered on the life of Kate Reddy, a...",89,Comedy,The Weinstein Company,9/16/2011,5.2,2011
1608,tt0330793,33000000,54700105,The Punisher,Thomas Jane|John Travolta|Will Patton|Roy Sche...,Jonathan Hensleigh,"There is no justice, there is only revenge.",When undercover FBI agent Frank Castle's wife ...,124,Action|Crime|Drama,Lions Gate Films|VIP 3 Medienfonds|Valhalla Mo...,4/15/2004,6.0,2004
1241,tt1815862,130000000,243843127,After Earth,Jaden Smith|Will Smith|Sophie Okonedo|ZoÃ« Kra...,M. Night Shyamalan,"Danger is real, fear is a choice",One thousand years after cataclysmic events fo...,100,Science Fiction|Action|Adventure,Columbia Pictures|Blinding Edge Pictures|Overb...,5/30/2013,5.1,2013
556,tt0252501,31000000,24185781,Hearts in Atlantis,Anthony Hopkins|Anton Yelchin|Hope Davis|Mika ...,Scott Hicks,What if one of life's great mysteries moved in...,A widowed mother and her son change when a mys...,101,Drama|Mystery,Castle Rock Entertainment,9/7/2001,6.3,2001


In [408]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [409]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
#Changed to timedelta format (to avoid date'1970-01-01' as in datetime)
#data['runtime'] = pd.to_timedelta(data['runtime'], unit='m')
#непонятно, зачем нужны действия, описанные выше, если ответы требуются в минутах, т.е. именно так, как в исходном варианте

#добавим столбец с прибылью profit: прибыль = сборы - бюджет
data['profit'] = data['revenue']-data['budget']
#добавим столбец title_len, содержащий длину названий фильмов
data['title_len'] = data['original_title'].map(lambda s: len(s))
#добавим столбец overview_words, содержащий длину(кол-во слов) описания фильмов
data['overview_len'] = data['overview'].map(lambda s: len(s.split()))

#некоторые столбцы в датасете содержат группу значений, где значения отделены друг от друга символом "|".
#для удобного анализа данных создадим  дополнительные столбцы со списком этих значений. 
#при необходимости "учета" каждого отдельного элемента из списка будем применять к этим столбцам функцию explode
#добавим столбец со списком режисеров
data['dir_list'] = data.director.str.split('|')
#добавим столбец со списком жанров
data['genres_list'] = data.genres.str.split('|')
#добавим столбец со списком актеров
data['cast_list'] = data.cast.str.split('|')
#добавим столбец со списком кинокомпаний
data['companies_list'] = data.production_companies.str.split('|')

data.head(2)


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,title_len,overview_len,dir_list,genres_list,cast_list,companies_list
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,...,6/9/2015,6.5,2015,1363528810,14,26,[Colin Trevorrow],"[Action, Adventure, Science Fiction, Thriller]","[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...","[Universal Studios, Amblin Entertainment, Lege..."
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,...,5/13/2015,7.1,2015,228436354,18,110,[George Miller],"[Action, Adventure, Science Fiction, Thriller]","[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...","[Village Roadshow Pictures, Kennedy Miller Pro..."


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [410]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' # +
# если ответили верно, можете добавить комментарий со значком "+"

In [411]:
# тут пишем ваш код для решения данного вопроса:
max_budget = data.loc[data['budget']==data['budget'].max()][['original_title','budget']]
max_budget

,original_title,budget
723,Pirates of the Caribbean: On Stranger Tides,380000000


ВАРИАНТ 2

# 2. Какой из фильмов самый длительный (в минутах)?

In [412]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)' # +

In [413]:
#интересно, зачем выше указано перевести runtime в datetime формат, 
#если ответы по этому столбцу все равно требуются в минутах???
longest_movie = data.loc[data['runtime']==data['runtime'].max()][['original_title', 'runtime']]
longest_movie

,original_title,runtime
1157,Gods and Generals,214


# 3. Какой из фильмов самый короткий (в минутах)?





In [414]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' # +

In [415]:
shortest_movie = data.loc[data['runtime']==data['runtime'].min()][['original_title', 'runtime']]
shortest_movie

,original_title,runtime
768,Winnie the Pooh,63


# 4. Какова средняя длительность фильмов?


In [416]:
answers['4'] = '2. 110' # +

In [417]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [418]:
answers['5'] = '1. 107' # +

In [419]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [420]:
# лучше код получения столбца profit вынести в Предобработку что в начале
#код data['profit'] = data['revenue']-data['budget'] вынесен в предобработку

In [421]:
answers['6'] = '5. Avatar (tt0499549)' # +

In [422]:
max_profit = data[data['profit']==data.profit.max()][['original_title', 'profit']]
max_profit

,original_title,profit
239,Avatar,2544505847


# 7. Какой фильм самый убыточный? 

In [423]:
answers['7'] = '5. The Lone Ranger (tt1210819)' # +

In [424]:
min_profit = data[data['profit']==data.profit.min()][['original_title', 'profit']]
min_profit

,original_title,profit
1245,The Lone Ranger,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [425]:
answers['8'] = '1. 1478' # +

In [426]:
pos_profit = data[data['profit']>0]
len(pos_profit)

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [427]:
answers['9'] = '4. The Dark Knight (tt0468569)'  # +

In [428]:
#создадим отдельный датафрейм с фильмами 2008 года выпуска
data_year2008 = data[data['release_year']==2008]
#найдем самый кассовый фильм года
best_revenue = data_year2008[data_year2008['revenue']==data_year2008['revenue'].max()][['original_title', 'revenue']]
best_revenue

,original_title,revenue
599,The Dark Knight,1001921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [429]:
answers['10'] = '5. The Lone Ranger (tt1210819)'  # +

In [430]:
#создадим отдельный датафрейм с фильмами 2012-2014 года выпуска
data_year12_14 = data[(data['release_year']<=2014) & (data['release_year']>=2012)]
#найдем самый убыточный фильм среди фильмов указанного периода
worst_profit = data_year12_14[data_year12_14['profit']==data_year12_14['profit'].min()][['original_title', 'profit']]
worst_profit

,original_title,profit
1245,The Lone Ranger,-165710090


# 11. Какого жанра фильмов больше всего?

In [431]:
answers['11'] = '3. Drama' # +

In [432]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

#собираем все жанры в Series и применяем value_counts
pd.Series(data['genres'].str.cat(sep='|').split('|')).value_counts()



Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

ВАРИАНТ 2

In [433]:
#в предобработке создали отдельный столбец со списком жанров, чтобы использовать explode
#data['genres_list'] = data.genres.str.replace('|',' ').str.split(' ')
data_exploded_genres = data.explode('genres_list')
data_exploded_genres['genres_list'].value_counts()
        

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres_list, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [434]:
answers['12'] = '1. Drama' # +

In [435]:
#создадим отдельный датафрейм только с прибыльными фильмами (profit>0)
data_prof = data[data['profit']>0]
data_prof
#посчитаем, какого жанра фильмов в новом датафрейме больше всего
pd.Series(data_prof['genres'].str.cat(sep='|').split('|')).value_counts()


Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [436]:
answers['13'] = '5. Peter Jackson' # +

In [437]:
#найдем сумму кассовых сборов для каждого режисера, применив функцию groupby
grouped_dir = data.groupby(['director'])['revenue'].sum().sort_values(ascending=False)
print(grouped_dir.head(2))

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
Name: revenue, dtype: int64


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [438]:
answers['14'] = '3. Robert Rodriguez' # +

In [439]:
#в предобработке добавим в исходный датафрейм отдельный столбец со списком режисеров
#data['dir_list'] = data.director.str.split('|')
#создадим отдельный датафрейм только для фильмов жанра Action
data_action = data[data.genres.str.contains("Action")]
#создадим новый датафрейм при помощи функции explode, чтобы иметь возможность "пройтись" по каждому режисеру, 
#если их несколько на 1 фильм
data_exploded_action = data_action.explode('dir_list')
#найдем режисера, который встречается в новом Action-датафрейме чаще всего
data_exploded_action.dir_list.value_counts()

Robert Rodriguez       9
Michael Bay            7
Paul W.S. Anderson     7
Antoine Fuqua          6
Ridley Scott           6
                      ..
Rob Minkoff            1
Julie Anne Robinson    1
Jay Russell            1
Des McAnuff            1
Joe Cornish            1
Name: dir_list, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [440]:
answers['15'] = '3. Chris Hemsworth' # +

In [441]:
grouped_actor = data.groupby(['director'])['revenue'].sum().sort_values(ascending=False)

#в предобработке добавим в исходный датафрейм отдельный столбец со списком актеров
#data['cast_list'] = data.cast.str.split('|')
#применим функцию explode, чтобы получить доступ к каждому актеру
data_exploded_cast = data.explode('cast_list')
#создадим отдельный датафрейм для фильмов 2012 года
data_2012 = data_exploded_cast[data_exploded_cast['release_year']==2012]
#применим groupby, чтобы посчитать сумму сборов всех фильмов с конкретным актером
grouped_cast = data_2012.groupby(['cast_list'])['revenue'].sum().sort_values(ascending=False)
grouped_cast


cast_list
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [442]:
answers['16'] = '3. Matt Damon' # +

In [443]:
#на основе датафрейма из предыдущего задания data_exploded_cast создадим новый, содержащий только высокобюджетные фильмы
data_high_budget = data_exploded_cast[data_exploded_cast['budget'] > data_exploded_cast['budget'].mean()]
#найдем самого популярного актера в высокобюджетных фильмах, используя groupby
top_actor = data_high_budget.groupby(['cast_list'])['imdb_id'].count().sort_values(ascending=False)
top_actor


cast_list
Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Eddie Murphy         15
Samuel L. Jackson    15
                     ..
Leslie Bibb           1
Leonard Nimoy         1
Lennie James          1
Lena Olin             1
50 Cent               1
Name: imdb_id, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [444]:
answers['17'] = '2. Action' # +

In [445]:
#на основе датафрейма data_exploded_cast создадим новый, содержащий только фильмы с Nicolas Cage
data_NC = data_exploded_cast[data_exploded_cast['cast_list']=='Nicolas Cage']
#применим explode, чтобы получить доступ ко всем жанрам в новом датафрейме
data_NC_exploded = data_NC.explode('genres_list')
#применим value_counts, чтобы найти самый популярный жанр с N.C.
NC_genre = data_NC_exploded.genres_list.value_counts()
NC_genre
                        


Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Animation           3
Mystery             3
Family              3
History             2
Romance             1
Horror              1
War                 1
Name: genres_list, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [446]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)' # +

In [447]:
#в предобработке добавим в исходный датафрейм отдельный столбец со списком компаний
#data['companies_list'] = data.production_companies.str.split('|')
#применим функцию explode, чтобы учитывать каждую компанию по-отдельности
data_exploded_companies = data.explode('companies_list')
#создадим отдельный датафрейм для фильмов Paramount Pictures
data_PP = data_exploded_companies[data_exploded_companies['companies_list']=='Paramount Pictures']
#найдем фильм от Paramount Pictures с наименьшей прибылью (profit).
unprof_movie = data_PP[data_PP['profit']==data_PP.profit.min()][['original_title', 'profit']]
unprof_movie



,original_title,profit
925,K-19: The Widowmaker,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [448]:
answers['19'] = '5. 2015' # +

In [449]:
year_max_revenue = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)
year_max_revenue

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [450]:
answers['20'] = '1. 2014' # +

In [451]:
#на основе созданного выше (задание 18) датафрейма с exploded кинокомпаниями сделаем датафрейм только для Warner Bros
data_WB = data_exploded_companies[data_exploded_companies.companies_list.str.match('Warner Bros')]
#при помощи groupby найдем самый прибыльный год
WB_max_profit_year = data_WB.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)
WB_max_profit_year

release_year
2014    2292949646
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1844008221
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1282142579
2012    1258020056
2002    1022709901
2015     870368348
2006     625351872
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [452]:
answers['21'] = '4. Сентябрь' # +

In [453]:
#из колонки release_date при помощи split() выделим только месяц 
#применим value_counts и найдем "самый популярный" месяц выпуска фильмов
months_only = data.release_date.apply(lambda s: s.split('/')[0]).value_counts()
months_only


9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: release_date, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [454]:
answers['22'] = '2. 450' # +

In [455]:
#создадим серию с только летними фильмами на основе созданной выше серии months_only
summer_movies = months_only.loc[['6','7','8']]
#посчитаем сумму фильмов, выпущенных летом
summer_movies.sum()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [456]:
answers['23'] = '5. Peter Jackson' # +

In [457]:
#создадим "зимний" датафрейм
data_winter = data[(data.release_date.str.match('11')) | (data.release_date.str.match('12')) | (data.release_date.str.match('1/'))]
#"достанем" каждого режисера из группы при помощи explode
data_winter_exploded_dirs = data_winter.explode('dir_list')
#найдем самого частовстречающегося в "зимнем" датафрейме режисера
data_winter_exploded_dirs.dir_list.value_counts()

#немного запутана формулировкой вопроса. 
#Подозреваю, что нужно было все же сравнивать, как много фильмов снимает каждый режисер в каждое время года.
#Ну и в итоге найти, у кого зимой выпущено больше фильмов в сравнении с остальными сезонами.


Peter Jackson        8
Clint Eastwood       7
Steven Soderbergh    6
Nancy Meyers         4
Edward Zwick         4
                    ..
Troy Nixey           1
Kevin Allen          1
Justin Chadwick      1
Rajkumar Hirani      1
Simon West           1
Name: dir_list, Length: 371, dtype: int64

In [458]:
answers['24'] = '5. Four By Two Productions' # +

#24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [459]:
#добавим в предобработку столбец title_len, содержащий длину названий фильмов
#data['title_len'] = data['original_title'].map(lambda s: len(s))
#применим функцию explode, чтобы учитывать каждую студию по-отдельности
data_exploded_companies_24 = data.explode('companies_list')
#при помощи groupby найдем студию, которая дает фильмам самые длинные в среднем названия
long_names_company = data_exploded_companies_24.groupby(['companies_list'])['title_len'].mean().sort_values(ascending=False)
long_names


companies_list
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Global Entertainment Group     2.0
Name: title_len, Length: 1771, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [460]:
answers['25'] = '3. Midnight Picture Show' # +

In [461]:
#добавим в предобработке столбец overview_words, содержащий длину(кол-во слов) описания фильмов
#data['overview_len'] = data['overview'].map(lambda s: len(s.split()))
#применим функцию explode, чтобы учитывать каждую студию по-отдельности
data_exploded_companies_25 = data.explode('companies_list')
#и при помощи groupby найдем студию, которая дает фильмам самые длинные в среднем описания
long_overviews_company = data_exploded_companies_25.groupby(['companies_list'])['overview_len'].mean().sort_values(ascending=False)
long_overviews_company


companies_list
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
98 MPH Productions                       159.0
Heineken Branded Entertainment           159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: overview_len, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [462]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave' # +

In [464]:
#выберем фильмы, чей рейтинг выше, чем рейтинг 99% фильмов, при помощи quantile
best_1 = data[data['vote_average'] > data['vote_average'].quantile(0.99)][['original_title','vote_average']]
best_1


,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [465]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint' # +

In [466]:
#пройдем по списку актеров каждого фильма и создадим все возможные пары актеров, снимавшихся вместе, 
#используя функцию combinations из itertools
#сохраним все пары в списке pairs_list
#создадим Series из pairs_list и при помощи value_counts найдем самые частовстречающиеся пары
pairs_list=[] 
for actors_list in data.cast_list:
    for pair in combinations(actors_list, 2):
        pairs_list.append(' '.join(sorted(pair)))

pairs_list = pd.Series(combo_list)
pairs_list.value_counts()



Emma Watson Rupert Grint            8
Daniel Radcliffe Rupert Grint       8
Daniel Radcliffe Emma Watson        8
Ben Stiller Owen Wilson             6
Helena Bonham Carter Johnny Depp    6
                                   ..
Joey King Maggie Gyllenhaal         1
Andrew Hampton Dax Shepard          1
Amy Sedaris Zach Galifianakis       1
Kevin Sorbo Shane Harper            1
Jamie Foxx Will Smith               1
Length: 17943, dtype: int64

ВАРИАНТ 2

# Submission

In [467]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [468]:
# и убедиться что ни чего не пропустил)
len(answers)

27